# 

# Import

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

import unicodedata # pour enlever les accents

# 1. Import de notre selection

In [ ]:
selection = pd.read_csv("Etape_3_selection.csv") #join.csv est notre dataset final

# 2. Préparation du model

## 2.1. Choix de X et uniformisation

In [ ]:
X = selection[selection.select_dtypes("number").columns]
scaler = StandardScaler()
scaler.fit(X)
X_scaler = scaler.transform(X)
X_scaler = pd.DataFrame(X_scaler, columns=selection.select_dtypes("number").columns)

## 2.2. Entraînement  du model

In [ ]:
# Nous l'initialisons et l'entraînons pour rechercher les 6 voisins

distance = NearestNeighbors(n_neighbors=6).fit(X_scaler)

## 2.3. Fonctions pour l'automatisation et la sortie des résultats

### 2.3.1. Fonction d'affichage

In [ ]:
def display(search, model, data, x_scale, titre):
    """
    input: search : array of funded index
           model : trained model
           data : the dataframe with movies
           x_scale : X value
           titre : title from input user
    """
    x_scaler = x_scale
    title = titre
    search_index = search.index
    
    if len(search) == 1:
        proches_title = model.kneighbors(x_scaler.iloc[search_index])
        print(f"""\nLes 5 films que vous nous recommandons pour \
\"{data.iloc[search_index[0]]['title_fr']}\" sorti en \
{int(data.iloc[search_index]['startYear'])} :\n""")
        
        for x in range(1, len(proches_title[1][0])):
            result = data.iloc[proches_title[1][0][x]]
            print(f'\tFilm numéro {x} :', result.loc["title_fr"] if result.loc["title_fr"] != '~'
                  else result.loc["originalTitle"], f'de {int(result.loc["startYear"])}')
            print()
        return True
    elif len(search) > 1:
        print(f"\nNous avons trouvé {len(search)} films indiquez le film souhaité :\n")

        for choix in range(len(search)):
            print(f"""Choix {choix + 1} : {data.iloc[search.index[choix]]['title_fr']} \
sorti en {int(data.iloc[search.index[choix]]['startYear'])}""")
            
        choix_user = int(input(f"\nVeuillez sélectionner un film entre 1 et {len(search)} : "))
        # Forget verification on choix_user
        
        proches_title = model.kneighbors([x_scaler.iloc[search_index[choix_user - 1]]])
        print(f"""\nLes 5 films que vous nous recommandons pour \
\"{search.iloc[choix_user - 1]['title_fr']}\" de {int(search.iloc[choix_user - 1]['startYear'])} : \n""")
      
        for x in range(1, len(proches_title[1][0])):
            result = data.iloc[proches_title[1][0][x]]
            print(f'\tFilm numéro {x} :', result.loc["title_fr"] if result.loc['title_fr'] != '~'
else f"{result.loc['originalTitle']} (Titre en VO)",f'de {int(result.loc["startYear"])} ')
            print()
        return True
    else:
        return False
        

### 2.3.2. Fonction pour avoir des recommandations

In [ ]:
def reco_film(data, x_scaler, model):
    """ 
        data = df selection de films features
        x_scaler = Features scaled
        model = Trained model
        return = nom 5 films proches voisins 
    
    """
    title_input = input("Veuillez entrer un titre de film complet : ")
    distance = model
    # Transform input to lower, strip and without accent
    
    nfkd_form = unicodedata.normalize('NFKD', title_input).strip()
    title = u"".join([c for c in nfkd_form if not unicodedata.combining(c)]).strip().lower()
    search = data.loc[data['title'] == title]
    if len(search) > 0:
        if(display(search, distance, data, x_scaler, title)):
            return f"Toute l'équipe de WildSalto vous souhaite Bon visionnage !!"

    else:
        search_2 = data.loc[data['title'].str.contains(title)]
        if(display(search_2, distance, data, x_scaler, title)):
            return f"Toute l'équipe de WildSalto vous souhaite Bon visionnage !!"
        else:
            return str(f'Le film "{title}" ne se trouve pas dans notre sélection :(')

In [ ]:
reco_film(data=selection, x_scaler=X_scaler, model=distance)

Veuillez entrer un titre de film complet : test

Nous avons trouvé 28 films indiquez le film souhaité :

Choix 1 : Testaments de femmes sorti en 1960
Choix 2 : Frank's Greatest Adventure sorti en 1967
Choix 3 : Le testament de l'Agha sorti en 1967
Choix 4 : La contestation sorti en 1969
Choix 5 : Contestation générale sorti en 1970
Choix 6 : Don Camillo et les contestataires sorti en 1972
Choix 7 : Screen Tests sorti en 1977
Choix 8 : Le dernier testament sorti en 1983
Choix 9 : Le testament du professeur Dowell sorti en 1984
Choix 10 : Le test sorti en 1987
Choix 11 : Le testament du soir sorti en 1995
Choix 12 : Le test de dureté sorti en 1998
Choix 13 : Teste di quoio sorti en 1981
Choix 14 : Le dernier testament sorti en 2001
Choix 15 : Je déteste la St-Valentin sorti en 2009
Choix 16 : Je déteste les enfants des autres sorti en 2007
Choix 17 : Le Testament de Grand-Mère sorti en 2020
Choix 18 : The Beta Test sorti en 2021
Choix 19 : Le test sorti en 2021
Choix 20 : The Greatest Sh

"Toute l'équipe de WildSalto vous souhaite Bon visionnage !!"